# ResearchArcade Complete Tutorial

This tutorial demonstrates how to work with the ResearchArcade database, covering all node types and edge relationships.

## Table of Contents
1. [Setup](#setup)
2. [OpenReview Data](#openreview)
3. [ArXiv Papers](#arxiv-papers)
4. [ArXiv Authors](#arxiv-authors)
5. [ArXiv Categories](#arxiv-categories)
6. [ArXiv Figures](#arxiv-figures)
7. [ArXiv Tables](#arxiv-tables)
8. [ArXiv Sections](#arxiv-sections)
9. [ArXiv Paragraphs](#arxiv-paragraphs)
10. [Relationships/Edges](#relationships)
11. [Advanced Queries](#advanced-queries)

## 1. Setup <a name="setup"></a>

In [54]:
import sys
from pathlib import Path
from tqdm import tqdm
import os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))
from research_arcade.research_arcade import ResearchArcade
import pandas as pd
from datetime import datetime

### Choose Database Backend

#### CSV Based

In [2]:
db_type = "csv"
config = {
    "csv_dir": "../data/my_research_arcade_data/"
}

research_arcade = ResearchArcade(db_type=db_type, config=config)

## 3. ArXiv Papers <a name="arxiv-papers"></a>

### Table Schema
- `id` (SERIAL PK)
- `arxiv_id` (VARCHAR, unique) - e.g., 1802.08773v3
- `base_arxiv_id` (VARCHAR) - e.g., 1802.08773
- `version` (INT) - e.g., 3
- `title` (TEXT)
- `abstract` (TEXT)
- `submit_date` (DATE)
- `metadata` (JSONB)

### Construct Table from API

In [3]:
config = {"arxiv_ids": ["1806.08804v4", "1903.03894v4"], "dest_dir": "./download"}
research_arcade.construct_table_from_api("arxiv_papers", config)

### Insert a Paper

In [4]:
# Example 1: Insert the famous "Attention is All You Need" paper
new_paper = {
    'arxiv_id': '1706.03762v7',
    'base_arxiv_id': '1706.03762',
    'version': 7,
    'title': 'Attention Is All You Need',
    'abstract': 'The dominant sequence transduction models are based on complex recurrent or convolutional neural networks in an encoder-decoder configuration. The best performing models also connect the encoder and decoder through an attention mechanism. We propose a new simple network architecture, the Transformer, based solely on attention mechanisms, dispensing with recurrence and convolutions entirely. Experiments on two machine translation tasks show these models to be superior in quality while being more parallelizable and requiring significantly less time to train. Our model achieves 28.4 BLEU on the WMT 2014 English-to-German translation task, improving over the existing best results, including ensembles by over 2 BLEU. On the WMT 2014 English-to-French translation task, our model establishes a new single-model state-of-the-art BLEU score of 41.8 after training for 3.5 days on eight GPUs, a small fraction of the training costs of the best models from the literature. We show that the Transformer generalizes well to other tasks by applying it successfully to English constituency parsing both with large and limited training data.',
    'submit_date': '2017-06-12',
    'metadata': {'venue': 'NeurIPS 2017', 'pdf_url': 'https://arxiv.org/pdf/1706.03762.pdf'}
}

research_arcade.insert_node("arxiv_papers", node_features=new_paper)
print("Paper inserted successfully!")

Paper inserted successfully!


In [5]:
# Example 2: Insert BERT paper
bert_paper = {
    'arxiv_id': '1810.04805v2',
    'base_arxiv_id': '1810.04805',
    'version': 2,
    'title': 'BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding',
    'abstract': 'We introduce a new language representation model called BERT, which stands for Bidirectional Encoder Representations from Transformers. Unlike recent language representation models, BERT is designed to pre-train deep bidirectional representations from unlabeled text by jointly conditioning on both left and right context in all layers. As a result, the pre-trained BERT model can be fine-tuned with just one additional output layer to create state-of-the-art models for a wide range of tasks, such as question answering and language inference, without substantial task-specific architecture modifications.',
    'submit_date': '2018-10-11',
    'metadata': {'venue': 'NAACL 2019', 'citations': 50000}
}

research_arcade.insert_node("arxiv_papers", node_features=bert_paper)
print("BERT paper inserted successfully!")

BERT paper inserted successfully!


### Get All Papers

In [6]:
arxiv_papers_df = research_arcade.get_all_node_features("arxiv_papers")
print(f"Total papers in database: {len(arxiv_papers_df)}")
print("\nFirst 5 papers:")
print(arxiv_papers_df.head())

Total papers in database: 6

First 5 papers:
   id      arxiv_id  base_arxiv_id  version  \
0   2  1810.04805v2     1810.04805        2   
1   3   1409.0473v7     1409.04730        7   
2   4  1512.03385v1     1512.03385        1   
3   5  1806.08804v4     1806.08804        4   
4   6  1903.03894v4     1903.03894        4   

                                               title  \
0  BERT: Pre-training of Deep Bidirectional Trans...   
1  Neural Machine Translation by Jointly Learning...   
2       Deep Residual Learning for Image Recognition   
3  Hierarchical Graph Representation Learning wit...   
4  GNNExplainer: Generating Explanations for Grap...   

                                            abstract  \
0  We introduce a new language representation mod...   
1  Neural machine translation is a recently propo...   
2  Deeper neural networks are more difficult to t...   
3  Recently, graph neural networks (GNNs) have re...   
4  Graph Neural Networks (GNNs) are a powerful to...   

### Get Specific Paper by ID

In [7]:
paper_id = {"arxiv_id": "1810.04805v2"}
paper_features = research_arcade.get_node_features_by_id("arxiv_papers", paper_id)
print("Paper details:")
print(paper_features.to_dict(orient="records")[0])

Paper details:
{'id': 2, 'arxiv_id': '1810.04805v2', 'base_arxiv_id': 1810.04805, 'version': 2, 'title': 'BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding', 'abstract': 'We introduce a new language representation model called BERT, which stands for Bidirectional Encoder Representations from Transformers. Unlike recent language representation models, BERT is designed to pre-train deep bidirectional representations from unlabeled text by jointly conditioning on both left and right context in all layers. As a result, the pre-trained BERT model can be fine-tuned with just one additional output layer to create state-of-the-art models for a wide range of tasks, such as question answering and language inference, without substantial task-specific architecture modifications.', 'submit_date': '2018-10-11', 'metadata': '{"venue": "NAACL 2019", "citations": 50000}'}


### Update a Paper

In [8]:
# Update metadata for a paper
updated_paper = {
    'arxiv_id': '1706.03762v7',
    'metadata': {
        'venue': 'NeurIPS 2017',
        'pdf_url': 'https://arxiv.org/pdf/1706.03762.pdf',
        'citations': 75000,
        'influential': True
    }
}

research_arcade.update_node("arxiv_papers", node_features=updated_paper)
print("Paper updated successfully!")

Paper updated successfully!


### Delete a Paper

In [9]:
# Delete a paper by ID
paper_id = {"arxiv_id": "1706.03762v7"}
deleted_paper = research_arcade.delete_node_by_id("arxiv_papers", paper_id)
print("Deleted paper:")
print(deleted_paper)

Deleted paper:
True


## 4. ArXiv Authors <a name="arxiv-authors"></a>

### Table Schema
- `id` (SERIAL PK)
- `semantic_scholar_id` (VARCHAR, unique)
- `name` (VARCHAR)
- `homepage` (VARCHAR)

### Construct Table from API

In [10]:
# config = {"arxiv_ids": ["1903.03894v4", "1806.08804v4"], "dest_dir": "./download"}
# research_arcade.construct_table_from_api("arxiv_authors", config)

### Insert Authors

In [11]:
# Insert authors from the Transformer paper
authors = [
    {
        'semantic_scholar_id': 'ss_ashish_vaswani',
        'name': 'Ashish Vaswani',
        'homepage': 'https://scholar.google.com/citations?user=oR9sCGYAAAAJ'
    },
    {
        'semantic_scholar_id': 'ss_noam_shazeer',
        'name': 'Noam Shazeer',
        'homepage': 'https://scholar.google.com/citations?user=oR9sCGYAAAAJ'
    },
    {
        'semantic_scholar_id': 'ss_niki_parmar',
        'name': 'Niki Parmar',
        'homepage': 'https://scholar.google.com/citations?user=oR9sCGYAAAAJ'
    },
    {
        'semantic_scholar_id': 'ss_jakob_uszkoreit',
        'name': 'Jakob Uszkoreit',
        'homepage': 'https://scholar.google.com/citations?user=oR9sCGYAAAAJ'
    },
    {
        'semantic_scholar_id': 'ss_llion_jones',
        'name': 'Llion Jones',
        'homepage': 'https://scholar.google.com/citations?user=oR9sCGYAAAAJ'
    }
]

for author in authors:
    research_arcade.insert_node("arxiv_authors", node_features=author)
    print(f"Inserted author: {author['name']}")

Inserted author: Ashish Vaswani
Inserted author: Noam Shazeer
Inserted author: Niki Parmar
Inserted author: Jakob Uszkoreit
Inserted author: Llion Jones


### Get All Authors

In [12]:
authors_df = research_arcade.get_all_node_features("arxiv_authors")
print(f"Total authors in database: {len(authors_df)}")
print("\nAll authors:")
print(authors_df)

Total authors in database: 13

All authors:
    id semantic_scholar_id                 name  \
0    1   ss_ashish_vaswani       Ashish Vaswani   
1    2     ss_noam_shazeer         Noam Shazeer   
2    3      ss_niki_parmar          Niki Parmar   
3    4  ss_jakob_uszkoreit      Jakob Uszkoreit   
4    5      ss_llion_jones          Llion Jones   
5    6            83539859             Rex Ying   
6    7            40974349      Dylan Bourgeois   
7    8           145829303          Jiaxuan You   
8    9             2095762            M. Zitnik   
9   10             1702139          J. Leskovec   
10  11           143622465   Christopher Morris   
11  12           145201124            Xiang Ren   
12  13            49437682  William L. Hamilton   

                                             homepage  
0                           https://ashishvaswani.com  
1   https://scholar.google.com/citations?user=oR9s...  
2   https://scholar.google.com/citations?user=oR9s...  
3   https://schol

### Get Specific Author by ID

In [13]:
author_id = {"semantic_scholar_id": "ss_ashish_vaswani"}
author_features = research_arcade.get_node_features_by_id("arxiv_authors", author_id)
print("Author details:")
print(author_features)

Author details:
None


### Update an Author

In [14]:
updated_author = {
    'semantic_scholar_id': 'ss_ashish_vaswani',
    'homepage': 'https://ashishvaswani.com'
}

research_arcade.update_node("arxiv_authors", node_features=updated_author)
print("Author updated successfully!")

Author updated successfully!


## 5. ArXiv Categories <a name="arxiv-categories"></a>

### Table Schema
- `id` (SERIAL PK)
- `name` (VARCHAR, unique)
- `description` (TEXT)

### Insert From API

In [15]:
config = {"arxiv_ids": ["1903.03894v4", "1806.08804v4"], "dest_dir": "./download"}
research_arcade.construct_table_from_api("arxiv_categories", config)

{'id': '1903.03894v4', 'title': 'GNNExplainer: Generating Explanations for Graph Neural Networks', 'abstract': "Graph Neural Networks (GNNs) are a powerful tool for machine learning on\ngraphs.GNNs combine node feature information with the graph structure by\nrecursively passing neural messages along edges of the input graph. However,\nincorporating both graph structure and feature information leads to complex\nmodels, and explaining predictions made by GNNs remains unsolved. Here we\npropose GNNExplainer, the first general, model-agnostic approach for providing\ninterpretable explanations for predictions of any GNN-based model on any\ngraph-based machine learning task. Given an instance, GNNExplainer identifies a\ncompact subgraph structure and a small subset of node features that have a\ncrucial role in GNN's prediction. Further, GNNExplainer can generate consistent\nand concise explanations for an entire class of instances. We formulate\nGNNExplainer as an optimization task that max

### Insert Categories

In [16]:
categories = [
    {
        'name': 'cs.CL',
        'description': 'Computation and Language (Natural Language Processing)'
    },
    {
        'name': 'cs.LG',
        'description': 'Machine Learning'
    },
    {
        'name': 'cs.AI',
        'description': 'Artificial Intelligence'
    },
    {
        'name': 'cs.CV',
        'description': 'Computer Vision and Pattern Recognition'
    },
    {
        'name': 'stat.ML',
        'description': 'Machine Learning (Statistics)'
    }
]

for category in categories:
    research_arcade.insert_node("arxiv_categories", node_features=category)
    print(f"Inserted category: {category['name']}")

Inserted category: cs.CL
Inserted category: cs.LG
Inserted category: cs.AI
Inserted category: cs.CV
Inserted category: stat.ML


### Get All Categories

In [17]:
categories_df = research_arcade.get_all_node_features("arxiv_categories")
print(f"Total categories: {len(categories_df)}")
print("\nAll categories:")
print(categories_df)

Total categories: 7

All categories:
   id     name                                        description
0   1    cs.CL  Computation and Language (Natural Language Pro...
1   2    cs.LG                                   Machine Learning
2   3    cs.AI                            Artificial Intelligence
3   4    cs.CV            Computer Vision and Pattern Recognition
4   5  stat.ML                      Machine Learning (Statistics)
5   6    cs.NE                                                NaN
6   7    cs.SI                                                NaN


## 6. ArXiv Figures <a name="arxiv-figures"></a>

### Table Schema
- `id` (SERIAL PK)
- `paper_arxiv_id` (VARCHAR FK → papers.arxiv_id)
- `path` (VARCHAR)
- `caption` (TEXT)
- `label` (TEXT)
- `name` (TEXT)

### Insert Figures

In [18]:
# Insert figures for the Transformer paper
figures = [
    {
        'paper_arxiv_id': '1706.03762v7',
        'path': '/figures/transformer_architecture.png',
        'caption': 'The Transformer model architecture. The left side shows the encoder stack and the right side shows the decoder stack.',
        'label': 'fig:architecture',
        'name': 'Figure 1'
    },
    {
        'paper_arxiv_id': '1706.03762v7',
        'path': '/figures/scaled_dot_product_attention.png',
        'caption': 'Scaled Dot-Product Attention and Multi-Head Attention mechanisms.',
        'label': 'fig:attention',
        'name': 'Figure 2'
    },
    {
        'paper_arxiv_id': '1706.03762v7',
        'path': '/figures/positional_encoding.png',
        'caption': 'Positional encoding visualization showing sine and cosine functions of different frequencies.',
        'label': 'fig:positional',
        'name': 'Figure 3'
    }
]

for figure in figures:
    research_arcade.insert_node("arxiv_figures", node_features=figure)
    print(f"Inserted {figure['name']}")

Inserted Figure 1
Inserted Figure 2
Inserted Figure 3


### Get All Figures

In [19]:
figures_df = research_arcade.get_all_node_features("arxiv_figures")
print(f"Total figures: {len(figures_df)}")
print("\nAll figures:")
print(figures_df[['name', 'caption', 'label']])

Total figures: 135

All figures:
         name                                            caption  \
0    Figure 1  The Transformer model architecture. The left s...   
1    Figure 2  Scaled Dot-Product Attention and Multi-Head At...   
2    Figure 3  Positional encoding visualization showing sine...   
3         NaN  \caption{\gnn computation graph $G_c$ for maki...   
4         NaN  \caption{\name provides interpretable explanat...   
..        ...                                                ...   
130       NaN                                         \caption{}   
131       NaN                                         \caption{}   
132       NaN                                         \caption{}   
133       NaN                                         \caption{}   
134       NaN                                         \caption{}   

                                label  
0                    fig:architecture  
1                       fig:attention  
2                      fig:pos

## 7. ArXiv Tables <a name="arxiv-tables"></a>

### Table Schema
- `id` (SERIAL PK)
- `paper_arxiv_id` (VARCHAR FK → papers.arxiv_id)
- `path` (VARCHAR)
- `caption` (TEXT)
- `label` (TEXT)
- `table_text` (TEXT)

### Insert From API

In [20]:
config = {"arxiv_ids": ["1903.03894v4", "1806.08804v4"], "dest_dir": "./download"}
research_arcade.construct_table_from_api("arxiv_tables", config)

### Insert Categories

In [21]:
categories = [
    {
        'name': 'cs.CL',
        'description': 'Computation and Language (Natural Language Processing)'
    },
    {
        'name': 'cs.LG',
        'description': 'Machine Learning'
    },
    {
        'name': 'cs.AI',
        'description': 'Artificial Intelligence'
    },
    {
        'name': 'cs.CV',
        'description': 'Computer Vision and Pattern Recognition'
    },
    {
        'name': 'stat.ML',
        'description': 'Machine Learning (Statistics)'
    }
]

for category in categories:
    research_arcade.insert_node("arxiv_categories", node_features=category)
    print(f"Inserted category: {category['name']}")

Inserted category: cs.CL
Inserted category: cs.LG
Inserted category: cs.AI
Inserted category: cs.CV
Inserted category: stat.ML


### Get All Categories

In [22]:
categories_df = research_arcade.get_all_node_features("arxiv_categories")
print(f"Total categories: {len(categories_df)}")
print("\nAll categories:")
print(categories_df)

Total categories: 7

All categories:
   id     name                                        description
0   1    cs.CL  Computation and Language (Natural Language Pro...
1   2    cs.LG                                   Machine Learning
2   3    cs.AI                            Artificial Intelligence
3   4    cs.CV            Computer Vision and Pattern Recognition
4   5  stat.ML                      Machine Learning (Statistics)
5   6    cs.NE                                                NaN
6   7    cs.SI                                                NaN


## 6. ArXiv Figures <a name="arxiv-figures"></a>

### Table Schema
- `id` (SERIAL PK)
- `paper_arxiv_id` (VARCHAR FK → papers.arxiv_id)
- `path` (VARCHAR)
- `caption` (TEXT)
- `label` (TEXT)
- `name` (TEXT)

### Insert From API

In [23]:
config = {"arxiv_ids": ["1903.03894v4", "1806.08804v4"], "dest_dir": "./download"}
research_arcade.construct_table_from_api("arxiv_figures", config)

### Insert Tables

In [24]:
# Insert tables for the Transformer paper
tables = [
    {
        'paper_arxiv_id': '1706.03762v7',
        'path': '/tables/model_variations.tex',
        'caption': 'Variations on the Transformer architecture with different hyperparameters.',
        'label': 'tab:variations',
        'table_text': 'Model | N | d_model | d_ff | h | d_k | d_v | P_drop | train time\nbase | 6 | 512 | 2048 | 8 | 64 | 64 | 0.1 | 12 hrs'
    },
    {
        'paper_arxiv_id': '1706.03762v7',
        'path': '/tables/wmt_results.tex',
        'caption': 'Performance of the Transformer on WMT 2014 English-German and English-French translation tasks.',
        'label': 'tab:wmt',
        'table_text': 'Model | EN-DE BLEU | EN-FR BLEU\nTransformer (base) | 27.3 | 38.1\nTransformer (big) | 28.4 | 41.8'
    },
    {
        'paper_arxiv_id': '1706.03762v7',
        'path': '/tables/parsing_results.tex',
        'caption': 'English constituency parsing results on WSJ test set.',
        'label': 'tab:parsing',
        'table_text': 'Model | WSJ 23 F1\nTransformer | 91.3'
    }
]

for table in tables:
    research_arcade.insert_node("arxiv_tables", node_features=table)
    print(f"Inserted table: {table['label']}")

Inserted table: tab:variations
Inserted table: tab:wmt
Inserted table: tab:parsing


### Get All Tables

In [25]:
tables_df = research_arcade.get_all_node_features("arxiv_tables")
print(f"Total tables: {len(tables_df)}")
print("\nAll tables:")
print(tables_df[['label', 'caption']])

Total tables: 59

All tables:
                         label  \
0               tab:variations   
1                      tab:wmt   
2                  tab:parsing   
3   \label{fig:synth_datasets}   
4       \label{tab:results_pr}   
5       \label{tab:results_pr}   
6          \label{tab:results}   
7         \label{tab:results2}   
8               tab:variations   
9                      tab:wmt   
10                 tab:parsing   
11  \label{fig:synth_datasets}   
12      \label{tab:results_pr}   
13      \label{tab:results_pr}   
14         \label{tab:results}   
15        \label{tab:results2}   
16              tab:variations   
17                     tab:wmt   
18                 tab:parsing   
19  \label{fig:synth_datasets}   
20      \label{tab:results_pr}   
21      \label{tab:results_pr}   
22         \label{tab:results}   
23        \label{tab:results2}   
24              tab:variations   
25                     tab:wmt   
26                 tab:parsing   
27  \label{fig:syn

## 8. ArXiv Sections <a name="arxiv-sections"></a>

### Table Schema
- `id` (SERIAL PK)
- `content` (TEXT)
- `title` (TEXT)
- `appendix` (BOOLEAN)
- `paper_arxiv_id` (VARCHAR FK → papers.arxiv_id)
- `section_in_paper_id` (INT)

### Insert From API

In [26]:
config = {"arxiv_ids": ["1903.03894v4", "1806.08804v4"], "dest_dir": "./download"}
research_arcade.construct_table_from_api("arxiv_sections", config)

### Insert Sections

In [27]:
# Insert sections for the Transformer paper
sections = [
    {
        'content': 'The dominant sequence transduction models are based on complex recurrent or convolutional neural networks that include an encoder and a decoder...',
        'title': 'Introduction',
        'appendix': False,
        'paper_arxiv_id': '1706.03762v7',
        'section_in_paper_id': 1
    },
    {
        'content': 'Most competitive neural sequence transduction models have an encoder-decoder structure. Here, the encoder maps an input sequence of symbol representations...',
        'title': 'Background',
        'appendix': False,
        'paper_arxiv_id': '1706.03762v7',
        'section_in_paper_id': 2
    },
    {
        'content': 'Most neural sequence transduction models have an encoder-decoder structure. The Transformer follows this overall architecture using stacked self-attention and point-wise, fully connected layers...',
        'title': 'Model Architecture',
        'appendix': False,
        'paper_arxiv_id': '1706.03762v7',
        'section_in_paper_id': 3
    },
    {
        'content': 'In this section we describe the training regime for our models...',
        'title': 'Training',
        'appendix': False,
        'paper_arxiv_id': '1706.03762v7',
        'section_in_paper_id': 4
    },
    {
        'content': 'On the WMT 2014 English-to-German translation task, the big transformer model outperforms the best previously reported models...',
        'title': 'Results',
        'appendix': False,
        'paper_arxiv_id': '1706.03762v7',
        'section_in_paper_id': 5
    },
    {
        'content': 'In this work, we presented the Transformer, the first sequence transduction model based entirely on attention, replacing the recurrent layers...',
        'title': 'Conclusion',
        'appendix': False,
        'paper_arxiv_id': '1706.03762v7',
        'section_in_paper_id': 6
    }
]

for section in sections:
    research_arcade.insert_node("arxiv_sections", node_features=section)
    print(f"Inserted section: {section['title']}")

Inserted section: Introduction
Inserted section: Background
Inserted section: Model Architecture
Inserted section: Training
Inserted section: Results
Inserted section: Conclusion


### Get All Sections

In [28]:
sections_df = research_arcade.get_all_node_features("arxiv_sections")
print(f"Total sections: {sections_df}")
print("\nAll sections:")
print(sections_df[['title', 'section_in_paper_id', 'appendix']])

Total sections:       id                                            content  \
0      1  The dominant sequence transduction models are ...   
1      2  Most competitive neural sequence transduction ...   
2      3  Most neural sequence transduction models have ...   
3      4  In this section we describe the training regim...   
4      5  On the WMT 2014 English-to-German translation ...   
..   ...                                                ...   
141  142  Most competitive neural sequence transduction ...   
142  143  Most neural sequence transduction models have ...   
143  144  In this section we describe the training regim...   
144  145  On the WMT 2014 English-to-German translation ...   
145  146  In this work, we presented the Transformer, th...   

                  title  appendix paper_arxiv_id  section_in_paper_id  
0          Introduction     False   1706.03762v7                  1.0  
1            Background     False   1706.03762v7                  2.0  
2    Model 

## 9. ArXiv Paragraphs <a name="arxiv-paragraphs"></a>

### Table Schema
- `id` (SERIAL PK)
- `paragraph_id` (INT)
- `content` (TEXT)
- `paper_arxiv_id` (VARCHAR FK → papers.arxiv_id)
- `paper_section` (TEXT)
- `section_id` (INT)
- `paragraph_in_paper_id` (INT)

### Insert From API

In [29]:
config = {"arxiv_ids": ["1903.03894v4", "1806.08804v4"], "dest_dir": "./download"}
research_arcade.construct_table_from_api("arxiv_paragraphs", config)

100%|██████████| 2/2 [00:00<00:00, 146.23it/s]

1903.03894v4
Key to References: {'fig:explainer-motivation': 'figures_3', 'fig:explainer-intro': 'figures_4', 'fig:definition-node-features': 'figures_5', 'fig:including-node-features': 'figures_7', 'fig:subgraph_node': 'figures_8', 'fig:subgraph_graph': 'figures_9', 'fig:prototype': 'figures_12', 'fig:my_label': 'figures_11', 'fig:synth_datasets': 'table_13', 'tab:results_pr': 'table_15'}
No paper found for  cho2011friendship
No paper found for  you2018graph
No paper found for  zitnik2018decagon
No paper found for  zhang_deep_2018
No paper found for  zhou_graph_2018
No paper found for  graphsage
No paper found for  kipf2016semi
No paper found for  ying2018hierarchical
No paper found for  zhang2018link
No paper found for  doshi-velez_towards_2017
No paper found for  lakkaraju_interpretable_2017
No paper found for  ribeiro_why_2016
No paper found for  schmitz_ann-dt:_1999
No paper found for  chen2018learning
No paper found for  Erhan2009VisualizingHF
No paper found for  lundberg_unified

### Insert Paragraphs

In [30]:
# Insert paragraphs from the Introduction section
paragraphs = [
    {
        'paragraph_id': 1,
        'content': 'Recurrent neural networks, long short-term memory and gated recurrent neural networks in particular, have been firmly established as state of the art approaches in sequence modeling and transduction problems such as language modeling and machine translation.',
        'paper_arxiv_id': '1706.03762v7',
        'paper_section': 'Introduction',
        'section_id': 1,
        'paragraph_in_paper_id': 1
    },
    {
        'paragraph_id': 2,
        'content': 'Numerous efforts have since continued to push the boundaries of recurrent language models and encoder-decoder architectures. Recurrent models typically factor computation along the symbol positions of the input and output sequences.',
        'paper_arxiv_id': '1706.03762v7',
        'paper_section': 'Introduction',
        'section_id': 1,
        'paragraph_in_paper_id': 2
    },
    {
        'paragraph_id': 3,
        'content': 'Aligning the positions to steps in computation time, they generate a sequence of hidden states h_t, as a function of the previous hidden state h_{t-1} and the input for position t. This inherently sequential nature precludes parallelization within training examples, which becomes critical at longer sequence lengths, as memory constraints limit batching across examples.',
        'paper_arxiv_id': '1706.03762v7',
        'paper_section': 'Introduction',
        'section_id': 1,
        'paragraph_in_paper_id': 3
    },
    {
        'paragraph_id': 4,
        'content': 'Attention mechanisms have become an integral part of compelling sequence modeling and transduction models in various tasks, allowing modeling of dependencies without regard to their distance in the input or output sequences.',
        'paper_arxiv_id': '1706.03762v7',
        'paper_section': 'Introduction',
        'section_id': 1,
        'paragraph_in_paper_id': 4
    },
    {
        'paragraph_id': 5,
        'content': 'In this work we propose the Transformer, a model architecture eschewing recurrence and instead relying entirely on an attention mechanism to draw global dependencies between input and output. The Transformer allows for significantly more parallelization and can reach a new state of the art in translation quality after being trained for as little as twelve hours on eight P100 GPUs.',
        'paper_arxiv_id': '1706.03762v7',
        'paper_section': 'Introduction',
        'section_id': 1,
        'paragraph_in_paper_id': 5
    }
]

for paragraph in paragraphs:
    research_arcade.insert_node("arxiv_paragraphs", node_features=paragraph)
    print(f"Inserted paragraph {paragraph['paragraph_id']} from {paragraph['paper_section']}")

Inserted paragraph 1 from Introduction
Inserted paragraph 2 from Introduction
Inserted paragraph 3 from Introduction
Inserted paragraph 4 from Introduction
Inserted paragraph 5 from Introduction


### Get All Paragraphs

In [31]:
paragraphs_df = research_arcade.get_all_node_features("arxiv_paragraphs")
print(f"Total paragraphs: {len(paragraphs_df)}")
print("\nFirst 3 paragraphs:")
print(paragraphs_df[['paragraph_id', 'paper_section', 'content']].head(3))

Total paragraphs: 255

First 3 paragraphs:
   paragraph_id paper_section  \
0             1  Introduction   
1             2  Introduction   
2             3  Introduction   

                                             content  
0  Recurrent neural networks, long short-term mem...  
1  Numerous efforts have since continued to push ...  
2  Aligning the positions to steps in computation...  


## 10. Relationships/Edges <a name="relationships"></a>

This section demonstrates how to create and manage relationships between different entities.

### 10.2 ArXiv Citations (arxiv_citation)

#### Insert Citation

In [32]:
citation = {
    'citing_arxiv_id': '1810.04805v2',
    'cited_arxiv_id': '1706.03762v7',
    'bib_title': 'attention is all you need',
    'bib_key': 'something',
    'citing_sections': 'citing_sections',
}
research_arcade.insert_edge("arxiv_citation", edge_features=citation)
print("Citation created!")

Citation created!


#### Get All Citations

In [33]:
all_citations = research_arcade.get_all_edge_features("arxiv_citation")
print(f"Total citations: {len(all_citations)}")
print(all_citations.head())

Total citations: 1
   id citing_arxiv_id cited_arxiv_id                  bib_title    bib_key  \
0   1    1810.04805v2   1706.03762v7  attention is all you need  something   

   author_cited_paper    citing_sections citing_paragraphs  
0                 NaN  "citing_sections"                []  


#### Get Cited Papers

In [34]:
citing_paper = {'citing_paper_id': '1810.04805v2'}
cited_papers = research_arcade.get_neighborhood("arxiv_citation", primary_key=citing_paper)
print("Papers cited:")
print(cited_papers)

Papers cited:
   id citing_arxiv_id cited_arxiv_id                  bib_title    bib_key  \
0   1    1810.04805v2   1706.03762v7  attention is all you need  something   

   author_cited_paper    citing_sections citing_paragraphs  
0                 NaN  "citing_sections"                []  


#### Get Citing Papers

In [35]:
cited_paper = {'cited_paper_id': '1706.03762v7'}
citing_papers = research_arcade.get_neighborhood("arxiv_citation", primary_key=cited_paper)
print("Papers that cite:")
print(citing_papers)

Papers that cite:
   id citing_arxiv_id cited_arxiv_id                  bib_title    bib_key  \
0   1    1810.04805v2   1706.03762v7  attention is all you need  something   

   author_cited_paper    citing_sections citing_paragraphs  
0                 NaN  "citing_sections"                []  


#### Delete Citation

In [36]:
citation_id = {
    'citing_paper_id': '1810.04805v2',
    'cited_paper_id': '1706.03762v7'
}
research_arcade.delete_edge_by_id("arxiv_citation", primary_key=citation_id)
print("Citation deleted!")

Deleted citation: 1810.04805v2 -> 1706.03762v7
Citation deleted!


### 10.3 ArXiv Paper-Author (arxiv_paper_author)

#### Insert Paper-Author Relationships

In [37]:
paper_authors = [
    {'paper_arxiv_id': '1706.03762v7', 'author_id': 'ss_ashish_vaswani', 'author_sequence': 1},
    {'paper_arxiv_id': '1706.03762v7', 'author_id': 'ss_noam_shazeer', 'author_sequence': 2},
    {'paper_arxiv_id': '1706.03762v7', 'author_id': 'ss_niki_parmar', 'author_sequence': 3}
]
for relation in paper_authors:
    research_arcade.insert_edge("arxiv_paper_author", edge_features=relation)
    print(f"Linked author {relation['author_id']} (position {relation['author_sequence']})")

Linked author ss_ashish_vaswani (position 1)
Linked author ss_noam_shazeer (position 2)
Linked author ss_niki_parmar (position 3)


#### Get All Paper-Author Relationships

In [38]:
all_relations = research_arcade.get_all_edge_features("arxiv_paper_author")
print(f"Total relationships: {len(all_relations)}")
print(all_relations.head(10))

Total relationships: 3
  paper_arxiv_id          author_id  author_sequence
0   1706.03762v7  ss_ashish_vaswani                1
1   1706.03762v7    ss_noam_shazeer                2
2   1706.03762v7     ss_niki_parmar                3


#### Get Authors for a Paper

In [39]:
paper_id = {'paper_arxiv_id': '1706.03762v7'}
authors = research_arcade.get_neighborhood("arxiv_paper_author", primary_key=paper_id)
print("Authors:")
print(authors.sort_values('author_sequence'))

Authors:
  paper_arxiv_id          author_id  author_sequence
0   1706.03762v7  ss_ashish_vaswani                1
1   1706.03762v7    ss_noam_shazeer                2
2   1706.03762v7     ss_niki_parmar                3


#### Get Papers by Author

In [40]:
author_id = {'author_id': 'ss_ashish_vaswani'}
papers = research_arcade.get_neighborhood("arxiv_paper_author", primary_key=author_id)
print("Papers by author:")
print(papers)

Papers by author:
  paper_arxiv_id          author_id  author_sequence
0   1706.03762v7  ss_ashish_vaswani                1


#### Delete Paper-Author Link

In [41]:
relation_id = {'paper_arxiv_id': '1706.03762v7', 'author_id': 'ss_ashish_vaswani'}
research_arcade.delete_edge_by_id("arxiv_paper_author", primary_key=relation_id)
print("Relationship deleted!")

Relationship deleted!


### 10.4 ArXiv Paper-Category (arxiv_paper_category)

#### Insert Paper-Category Relationships

In [42]:
paper_categories = [
    {'paper_arxiv_id': '1706.03762v7', 'category_id': '1'},
    {'paper_arxiv_id': '1706.03762v7', 'category_id': '1'},
    {'paper_arxiv_id': '1706.03762v7', 'category_id': '2'}
]
for relation in paper_categories:
    research_arcade.insert_edge("arxiv_paper_category", edge_features=relation)
    print(f"Linked {relation['category_id']}")

Linked 1
Linked 1
Linked 2


#### Get All Paper-Category Relationships

In [43]:
all_relations = research_arcade.get_all_edge_features("arxiv_paper_category")
print(f"Total relationships: {len(all_relations)}")
print(all_relations.head())

Total relationships: 15
  paper_arxiv_id  category_id
0   1706.03762v7            1
1   1706.03762v7            2
2   1810.04805v2            1
3   1810.04805v2            3
4   1706.03762v7            1


#### Get Categories for Paper

In [44]:
paper_id = {'paper_arxiv_id': '1706.03762v7'}
categories = research_arcade.get_neighborhood("arxiv_paper_category", primary_key=paper_id)
print("Categories:")
print(categories)

Categories:
   paper_arxiv_id  category_id
0    1706.03762v7            1
1    1706.03762v7            2
2    1706.03762v7            1
3    1706.03762v7            1
4    1706.03762v7            1
5    1706.03762v7            1
6    1706.03762v7            2
7    1706.03762v7            1
8    1706.03762v7            1
9    1706.03762v7            2
10   1706.03762v7            1
11   1706.03762v7            1
12   1706.03762v7            2


#### Get Papers in Category

In [45]:
category_id = {'category_id': 'cs.LG'}
papers = research_arcade.get_neighborhood("arxiv_paper_category", primary_key=category_id)
print("Papers in category:")
print(papers)

Papers in category:
None


#### Delete Paper-Category Link

In [46]:
relation_id = {'paper_arxiv_id': '1706.03762v7', 'category_id': 'cs.AI'}
research_arcade.delete_edge_by_id("arxiv_paper_category", primary_key=relation_id)
print("Relationship deleted!")

Relationship deleted!


### 10.5 ArXiv Paper-Figure (arxiv_paper_figure)

#### Insert Paper-Figure Relationships

In [47]:
paper_figures = [
    {'paper_arxiv_id': '1706.03762v7', 'figure_id': 1},
    {'paper_arxiv_id': '1706.03762v7', 'figure_id': 2}
]
for relation in paper_figures:
    research_arcade.insert_edge("arxiv_paper_figure", edge_features=relation)
    print(f"Linked figure {relation['figure_id']})")

Linked figure 1)
Linked figure 2)


#### Get Figures for Paper

In [48]:
paper_id = {'paper_arxiv_id': '1706.03762v7'}
figures = research_arcade.get_neighborhood("arxiv_paper_figure", primary_key=paper_id)
print("Figures:")
print(figures)

Figures:
  paper_arxiv_id  figure_id
0   1706.03762v7          1
1   1706.03762v7          2


### 10.6 ArXiv Paper-Table (arxiv_paper_table)

#### Insert Paper-Table Relationships

In [49]:
paper_tables = [
    {'paper_arxiv_id': '1706.03762v7', 'table_id': 1},
    {'paper_arxiv_id': '1706.03762v7', 'table_id': 2}
]
for relation in paper_tables:
    research_arcade.insert_edge("arxiv_paper_table", edge_features=relation)
    print(f"Linked table {relation['table_id']}")

Linked table 1
Linked table 2


#### Get Tables for Paper

In [51]:
paper_id = {'paper_arxiv_id': '1706.03762v7'}
tables = research_arcade.get_neighborhood("arxiv_paper_table", primary_key=paper_id)
print("Tables:")


Tables:


### 10.7 ArXiv Paragraph-Reference (arxiv_paragraph_reference)

#### Insert Paragraph-Reference Relationships

In [52]:
paragraph_references = [
    {'paragraph_id': 1, 'paper_section': 'established approaches', 'paper_arxiv_id': '1706.03762v7', 'reference_label': "{something}", 'reference_type': 'figure'}
]

for relation in paragraph_references:
    research_arcade.insert_edge("arxiv_paragraph_reference", edge_features=relation)

#### Get References in Paragraph

In [53]:
paragraph_id = {'paragraph_id': 1}
references = research_arcade.get_neighborhood("arxiv_paragraph_reference", primary_key=paragraph_id)
print("References:")
print(references)

References:
   id  paragraph_id           paper_section paper_arxiv_id reference_label  \
0   1             1  established approaches   1706.03762v7     {something}   
1   2             1  established approaches   1706.03762v7     {something}   
2   3             1  established approaches   1706.03762v7     {something}   
3   4             1  established approaches   1706.03762v7     {something}   

  reference_type  
0         figure  
1         figure  
2         figure  
3         figure  


## Conclusion

This tutorial has covered:

1. Setting up the ResearchArcade database connection
2. Working with OpenReview data
3. CRUD operations for all ArXiv entity types:
   - Papers
   - Authors
   - Categories
   - Figures
   - Tables
   - Sections
   - Paragraphs
4. Creating relationships between entities:
   - Authorship
   - Citations
   - Paper-Category links
   - Paper-Figure/Table links
   - Paragraph-level references
5. Advanced querying patterns
6. Best practices for data validation

For more information, refer to the ResearchArcade documentation.